# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [3]:
import pandas as pd

## imports for Python, Pandas

In [4]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [67]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [69]:
# use normalization to create tables from nested element

df = pd.json_normalize(data, 'counties')
df

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [8]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,info.governor,state,shortname
0,Dade,12345,Rick Scott,Florida,FL
1,Broward,40000,Rick Scott,Florida,FL
2,Palm Beach,60000,Rick Scott,Florida,FL
3,Summit,1234,John Kasich,Ohio,OH
4,Cuyahoga,1337,John Kasich,Ohio,OH


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [500]:
# load json as string
data = json.load((open('data/world_bank_projects.json')))

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

### 1 Find the 10 countries with most projects

In [250]:
data = json.load((open('data/world_bank_projects.json')))

In [501]:
df = pd.json_normalize(data, 'sector',[ "country_namecode", 'countrycode'], errors='ignore') 
df['country_namecode'] = [i.split('!$!')[0] for i in df['country_namecode']] #remove " !$!ET "
df = df[df['country_namecode'].isin(['Africa', 
                                     'World', 
                                     'West Bank and Gaza', 'Pacific Islands', 
                                     'Middle East and North Africa',
                                     'East Asia and Pacific']) == False]
# df.index.to_list()
df =  df.groupby('country_namecode').count()\
        .sort_values('Name', ascending=False)\
        .reset_index()\
        .rename(columns={'Name':'Count'}).drop('countrycode', axis=1).head(10)
df

,country_namecode,Count
0,Republic of Indonesia,56
1,People's Republic of China,54
2,Socialist Republic of Vietnam,50
3,Republic of India,48
4,Federative Republic of Brazil,37
5,Nepal,33
6,People's Republic of Bangladesh,32
7,Republic of Mozambique,32
8,Kingdom of Morocco,31
9,Burkina Faso,31


### Find the top 10 major project themes (using column 'mjtheme_namecode')

In [502]:
data = json.load((open('data/world_bank_projects.json')))
df = pd.json_normalize(data,  ['mjtheme_namecode'], errors='ignore')
df = df[df['name'] != '']
df = df.groupby('name').count().sort_values('code', ascending=False).reset_index().rename(columns={'code':"Count"})
df.head(10)


,name,Count
0,Environment and natural resources management,223
1,Rural development,202
2,Human development,197
3,Public sector governance,184
4,Social protection and risk management,158
5,Financial and private sector development,130
6,Social dev/gender/inclusion,119
7,Trade and integration,72
8,Urban development,47
9,Economic management,33


###  Create a dataframe with the missing names filled in.

In [507]:
data = json.load((open('data/world_bank_projects.json')))
df = pd.json_normalize(data, ['mjtheme_namecode'], errors='ignore')
# get index rows with empty name
empty_name_codes = df[df['name'] == ''].drop('name', axis=1).to_dict()['code']
print('Missing values in', len(empty_name_codes.keys()), 'rows')

Missing values in 122 rows


In [510]:
def getName(code):
    # get category name by code 
    names = df[df['code'] == str(code)]
    value = [i for i in set(names['name']) if len(i) != 0]
    if value:
        return value[-1]
    else:
        # if value not found
        return None
    
for k, v in empty_name_codes.items(): 
    name = getName(v)
    df.iloc[k,1] = name
if df[df['name'] == ''].size > 0:
    print('There are some rows with missing values')
else:
    print("Updated")
    
    

Updated


In [511]:
df

,code,name
0,8,Human development
1,11,Environment and natural resources management
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration
...,...,...
1494,10,Rural development
1495,9,Urban development
1496,8,Human development
1497,5,Trade and integration
